In [1]:
# Enable autoreload of module
%load_ext autoreload
%autoreload 2

In [44]:
from utils import metrics
from torchvision import datasets, transforms
from IPython.display import display
from IPython.core.display import HTML
import torch
import itertools
import einops

In [4]:
# Load mnist dataset
mnist = datasets.MNIST("mnist-data", train=True, download=True)

In [13]:
# generator function to get images from mnist dataset with a specific label
def get_images_by_label(dataset, label):
    for image, image_label in dataset:
        if image_label == label:
            yield image

# Get all images with label 5 up to 30 images
images = list(itertools.islice(get_images_by_label(mnist, 5), 30))

In [15]:
display(images[0])

In [36]:
# Get a generated and a reference set of images as tensors
gen_range = range(0, 10)
ref_range = range(10, 20)

# image to tensor transformation
def to_tensor(image):
    # image is a (28, 28) tensor
    return transforms.functional.pil_to_tensor(image) / 255.0

def to_image(tensor):
    return transforms.functional.to_pil_image(tensor)

gen_images = torch.stack([to_tensor(images[i]) for i in gen_range])
ref_images = torch.stack([to_tensor(images[i]) for i in ref_range])

In [37]:
# show as an image again
display(to_image(gen_images[0]))

In [52]:
# Pairwise similarity
pairwise_sim = metrics.pairwise_ssim_distance(gen_images, ref_images)
most_similar = torch.unravel_index( pairwise_sim.argmax(), pairwise_sim.shape)
most_dissimilar = torch.unravel_index( pairwise_sim.argmin(), pairwise_sim.shape)

# Show the most similar image
display(HTML("<h3>Most similar images</h3>"))
display(to_image(gen_images[most_similar[0].item()]))
display(to_image(ref_images[most_similar[1].item()]))

# Show the most dissimilar image
display(HTML("<h3>Most dissimilar images</h3>"))
display(to_image(gen_images[most_dissimilar[0].item()]))
display(to_image(ref_images[most_dissimilar[1].item()]))


In [55]:
# metrics
metrics.compute_all_metrics(gen_images, ref_images)

{'lgan_mmd': tensor(1.2517),
 'lgan_cov': tensor(0.4000),
 'lgan_mmd_smp': tensor(-6.0070)}